<img src='https://weclouddata.com/wp-content/uploads/2016/11/logo.png' width='30%'>
-------------

<h3 align='center'> Applied Machine Learning Course - In-class Lab Week 3 </h3>
<h1 align='center'> Naive Bayes: Spam vs. Ham </h1>

<br>
<center align="left"> Developed by:</center>
<center align="left"> WeCloudData Academy </center>

**Packages Used:**

- Pandas
- NumPy
- Sklearn
- [NLTK](https://www.nltk.org/): a convenient Python library for natural language processing.
-------

**Install NLTK**

- Install the NLTK package by running this in your terminal: `pip install nltk`
- Download relevant NTLK data by running this in your terminal: `python -m nltk.downloader stopwords`

# Load The Dataset

In this exercise, we will be using the famous [email spam classification dataset](https://www.kaggle.com/uciml/sms-spam-collection-dataset#spam.csv) `spam.csv`.

This CSV file contains two columns only: the `label` that you wish to predict, and the actual message.

There are two possible labels:
1. `spam`: this message is a spam.
2. `ham`: this message is not a spam.

We will be using `NaiveBayes` to build this message spam classifier.

In addition to practicing this new model, we will be experimenting with various techniques to perform text feature engineering on this dataset.

---------

# $\Delta$ Explore the dataset

In [ ]:
import pandas as pd
df = pd.read_csv('spam.csv')
df.head()

# $\Delta$ Separate labels and data

In [ ]:
y = df['label'].values
X = df['message'].values

In [ ]:
import numpy as np
np.unique(y)

# $\Omega$ (Practice) Split data into 80% training and 20% testing 

In [ ]:
# TODO



## Naive Bayes classifier

Naive Bayes classifier is a generative classifier:
- We build a model of $P(x│y)$, which can be used to generate new samples for a given class $y$.
- We apply Bayes rule to get $P(y│x)$.
- The decision rule is therefore:

<center>
    \begin{align}
    \widehat{y} & =arg\max_t P(y=t│x) \\
                & =arg\max_t \dfrac{P(y=t)P(x│y=t)}{P(x)} \\
                & =arg\max_t P(y=t)P(x│y=t),
    \end{align}
</center>

  where $t\in \{\textit{"spam"}, \textit{"ham"}\}$.


- Assuming conditional independence among features:
<center>
    $P(x│y=t)=\Pi_{j=1}^pP(x_j│y=t)$
</center>

In Naive Bayes, there is no optimization via gradient descent; rather, we need to estimate the following probabilities:

1. **Class priors**: $P(y=t)$
2. **Class-conditional features**: $P(x_j│y=t)$

-----

# $\Omega$ (Practice) Estimating class priors: $P(y=t)$


Similar to the models we learned previously, all the parameter fitting or probability estimation tasks must be **conducted on the training data only**.

Basically, to estimate class priors $P(y=t)$, all we need to do is to find out the natural (prior) distribution of the two classes `spam` and `ham` in our dataset.

In [ ]:
# Step 1: collect the count of each label



In [ ]:
# Step 2: convert raw count into probabilities for each class



# $\Delta$ Estimating conditional probabilities of features: $P(x_j│y=t)$


## Define features $x_j$

The most straightforward definition of each feature $x_j$ is by defining it as each individual word in the email. 

For example, $P(\textit{"you"}│y=\textit{"spam"})$ represents the probability of seeing the word "**_you_**" **given the email is a spam message**.

## Split emails into words (tokenization)

For text-type data, we often needs to split the entire text into words (**tokens**), so we can apply techniques such as **bag-of-words** models. This can be easily done using NLTK's `sentence_tokenize` and `word_tokenize` functions.

> Question: can we just split the text by space to get all the words?

> Answer: it works in most cases, but would fail for corner cases such as punctuations. For example, the sentence `It is a nice day.` should be tokenized into `["It", "is", "a", "nice", "day", "."]` -- notice how the period `.` is a token on its own. If we split this sentence by white spaces, we would end up with `["It", "is", "a", "nice", "day."]` instead.

In [ ]:
# Step 1: define preprocessing function: message -> sentences -> words
import nltk
def preprocess(message):
    sentences = nltk.sent_tokenize(message) # split the email message into sentences
    
    tokens = []
    for sentence in sentences:
        sentence_tokens = nltk.word_tokenize(sentence) # split each sentence into a list of words
        tokens.extend([token.lower() for token in sentence_tokens]) # convert each word to lower cases
        
    return tokens

In [ ]:
# Step 2: for each email, we apply preprocess()
words_train = [preprocess(message) for message in msg_train]

In [ ]:
# Step 3: check the tokenization result
for i, words in enumerate(words_train):
    print(f'{msg_train[i]} -> {words}')

## $\Omega$ (Practice) Calculate $P(x_j│y=t)$

Once we define $x_j$ as each individual word, how can we calculate the conditional probability $P(x_j│y=t)$, which represents the likelihood of seeing the word $x_j$ appears in a message of class $t$ (i.e., $t$="spam" or "ham"). The most naive way to calculate this likelihood $P(x_j│y=t)$ is by using the raw frequency:

<center>
$P(x_j│y=t)=\dfrac{count(x_j, t)}{\Sigma_{k=1}^V{count(x_k, t)}}$
</center>

For example, $P(\textit{"you"}│y=\textit{"spam"})=\dfrac{count(\textit{"you"}, \textit{"spam"})}{\Sigma_{k=1}^V{count(x_k, \textit{"spam"})}}$

- $count(\textit{"you"}│y=\textit{"spam"})$ is the number of times the **word "_you_" occurs in all _spam_ emails.**
- $\Sigma_{k=1}^V{count(x_k, \textit{"spam"})}$ is the **total number of times** each word in the vocabulary occurs in all _spam_ emails.

In [ ]:
import numpy as np

def estimate_p_xy(label2cnt, words_train, y_train):
    # Step 1: calculate the raw word counts for each class, and stores them in `count_xy`
    # for example, `count_xy` would look seomthing like (the actual numbers would be different):
    # {
    #    "spam": {"the": 100, "and": 99, "you": 64, ...},
    #    "ham": {"the": 200, "and": 70, "you": 12, ...}
    # }
    # which means, in `spam` messages, the word "the" appears 100 times, and the word "and" appears 99 times, and so on.
    # In `ham` messages, the word "the" appears 200 times, and the word "and" appears 70 times, and so on.
    # Note: this `count_xy` should be a dictionary of dictionaries, where the top-level key is the class, "spam" or "ham",
    # and each key maps to a dictionary with word frequencies, calculated from messages belonging to that class.    
    # TODO
    count_xy = {}
    
    
    
    
    
    
    # Step 2: normalize the raw frequencies by the denomiator in the formula above.
    # i.e., normalize the raw count in `count_xy` by the total number of words in the messages of this class 
    # After this step, the values in `count_xy` should be all floats, and thus represents valid probabilities.
    # TODO
    
    
    
    
    
    
    
    return count_xy

In [ ]:
count_xy = estimate_p_xy(label2cnt=label2cnt, words_train=words_train, y_train=y_train)

## $\Omega$ (Practice) Print out the top 20 most frequent words for each class

Use your calculated `count_xy` above, print out the top 20 most frequent words for each class.

What do you see in these lists? Do the top words look meaningful, and thus would be helpful for differentiating different message classes?

In [ ]:
# TODO









# $\Delta$ Generate predictions

Once we have the following probabilities:
1. The prior probabilties of the classes, $P(y=t)$, calculated in step 6;
2. The conditional likelihoods, $P(x_j|y=t)$, calculated in step 7

We can now generate predictions for a given new message -- whether this is a spam or ham message. The decision rule for predicting a new message instance, vectorized as $x$ is a follows:

<center>
    \begin{align}
    \widehat{y} & =arg\max_t P(y=t│x) \\
                & =arg\max_t \dfrac{P(y=t)P(x│y=t)}{P(x)} \\
                & =arg\max_t P(y=t)P(x│y=t) \\
                & =arg\max_t P(y=t)\prod_j P(x_j│y=t),
    \end{align}
</center>

## $\Delta$ Use the decision above to geneate the prediction for each test instance

As demonstrated in the decision rule formula, we simply need to calculate $P(y=t|x)$, for $t \in \{spam, ham\}$ by replacing the probabilities $P(y=t)$ and $P(x_j|y=t)$ with the probabilities we calculated from previous steps.

In [ ]:
def predict_spam_or_ham(words_test, priors, count_xy):
    cls_probs = dict() # keeps track P(y=t|x)
    for label, label_prior in priors.items():
        prob = 1.0
        for word in words_test:
            p_xy = count_xy[label][word]
            prob *= p_xy
        
        cls_probs[label] = prob * label_prior
        # note: cls_probs[label] does not represent valid probabilities, 
        # as we take out the normalizaton probability $P(x)$ in the calculation
    
    # the class label that maximzes `cls_probs` is our prediction
    best_label, best_prob = max(cls_probs.items(), key=lambda x:x[1], reverse=True)[0]
    
    return best_label, best_prob

In [ ]:
predictions = []
probs = []
for message in msg_test:
    words_test = preprocess(message) # use the same preprocessing on the test email message
    predicted_label, label_prob = predict_spam_or_ham(words_test, priors, count_xy)
    predictions.append(predicted_label)
    probs.append(label_prob)

## $\Omega$ (Pratice) Evaluate mode performance

Once we got our predictions on the test set, we can conduct performance evaluation to get an idea of how good this Naive Bayes implementation is.

Hint: use sklearn's `classification_report` and `accuracy_score` as we saw in last week.

In [ ]:
from sklearn.metrics import classification_report, accuracy_score

def evaluate_prediction(predictions, y_test):
    # TODO: print out classification report and the overall accuracy score
    
    

In [ ]:
evaluate_prediction(predictions=predictions, y_test=y_test)

Do you think the performance you have right now is decent? If not, let's move on to the improvement steps below.

# $\Delta$ Improvements

We got a working Naive Bayes model, but it is not doing so great right now.

There are couple problems with using the raw frequence to represent $P(x_j│y=t)$:

1. Any unseen words in the dataset would have $P(x_j│y=t)=0$, and therefore $P(x│y=t)=\Pi_{j=1}^pP(x_j│y=t)=0$, which is problematic. We call these words in the test set which are not observed in training set as `out-of-vocabulary` (OOV) words.
  > Hint: inspect the `predictions` and `probs` you got from Step 6.2 and see how some of the `probs` are zero.

2. As we saw in the class, we normally would want to remove stopwords from the text as those words bear little information. For example, as we saw in 5.3, the top 20 words in either class, _"spam"_ or _"ham"_, are pretty much all stopwords and punctuations.

## $\Delta$ Improvement 1: Smoothing to tackle zero probability $P(x_j│y=t)=0$


An easy technique for smoothing is to assume any word has at least one occurrence:


<center>
$P(x_j│y=t)=\dfrac{count(x_j, t)+1}{\Sigma_{k=1}^V{count(x_k, t)}+|V|}$
</center>

Where $V$ is the set of distinct words in the training data. We usually call $V$ the vocabulary of the training data. ANd $|V|$ is the size of this vocabulary, i.e., the number of distinct words in the training data.



### $\Omega$ (Practice)  Improve `estimate_p_xy` by applying the "plus one" smoothing

Now let's improve how we calculate `count_xy` in Step 6 by applying the "plus one" smoothing in the formula above.

In [ ]:
# Step 1

# Redefine a smoothed probability estimation

import numpy as np

def estimate_p_xy_smoothed(label2cnt, words_train, y_train, bias, V):
    # Step 1: calculate the raw word counts for each class, and stores them in `count_xy`.
    # This should be the same as the Step 1 in the `estimate_p_xy` function in Step 5.2.
    # TODO
    
    
    
    
    # Step 2: normalize the raw frequencies using the formula above.
    # i.e., we should add 1 to the numerator and the vocabulary size to the denominator.
    # After this step, the values in `count_xy` should be all floats, and thus represents valid probabilities.  
    # TODO
    
    
    
    
    
    return count_xy


In [ ]:
def get_p_xy(count_xy, label, word, bias, V):
    # if the word is not in our collected `count_xy` (it is an oov word), 
    # we will return a default value 1/V
    return count_xy[label].get(word, bias/V)

In [ ]:
def predict_spam_or_ham_smoothed(words_test, priors, count_xy, bias, V):
    cls_probs = dict() # keeps track P(y=t|x)
    for label, label_prior in priors.items():
        prob = 1.0
        for word in words_test:
            # Instead of using `count_xy[label][word]` directly, we call this `get_p_xy` function to perform
            # smoothing for the words (possibly out of vocabulary) in test instances
            p_xy = get_p_xy(count_xy=count_xy, label=label, word=word, bias=bias, V=V)
            prob *= p_xy
        
        cls_probs[label] = prob * label_prior
    
    best_label, label_prob = sorted(cls_probs.items(), key=lambda x:x[1], reverse=True)[0]
    return best_label, label_prob

### $\Omega$ (Practice) Calculate the vocbulary size

Implement the `get_vocabulary_size` function below to get the vocabulary size, i.e., the number of distinct words in the **training data**.

In [ ]:
def get_vocabulary_size(words_train):
    # TODO: count the number of distinct words in words_train
    
    
    
    
    

### $\Delta$ Generate new predictions 

We can now generate predictions using our improved model. The major difference between this step and Step 6.1 is how we calculate `count_xy` and how we get `best_label` and `label_prob`.

In [ ]:
V = get_vocabulary_size(words_train)

count_xy = estimate_p_xy_smoothed(label2cnt=label2cnt, words_train=words_train, y_train=y_train, bias=1, V=V)
predictions = []
for message in msg_test:
    words_test = preprocess(message) # use the same preprocessing procedure on the new test message `msg_test`
    best_label, label_prob = predict_spam_or_ham_smoothed(words_test, priors, count_xy, bias=1, V=V)
    predictions.append(best_label)

### $\Delta$ Evaluate your new model

Is it much better now?

In [ ]:
# TODO: evaluate your new model by comparing the new set of predictions with y_test





## $\Delta$ Improvement 2: Removing stopwords and punctuations

As introduced eariler, the second improvement we can work on is to remove stopwords and punctuations when we perform feature engineering. This step is generally the best practice for building NLP applications, although its effect sometimes depends on particular tasks. For example, for our spam vs. ham problem, intuitively, massive use of punctuations, especially `!`, in a message might be an indicator of spamming.

However, to answer the question of whether a certain feature engineering technique is useful or not, we need to implement it first, and observe how it affects the overall performance.

In this practice, We will use NLTK's stopwords set to look up English stopwords.

### $\Delta$ Define the list of exclude words: stopwords 

As introduced in the lecture, stopwords are those words which generally bear very little meaning, such as `a`, and `the`.

In [ ]:
from nltk.corpus import stopwords
en_stopwords = set(stopwords.words('english'))
print(en_stopwords)

 ### $\Delta$ Modify the `preprocess` function in Section 7 to exclude stopwords and punctuations

In [ ]:
# Step 1: define preprocessing function: message -> sentences -> words
import nltk
def preprocess_stopwords_removed(message, en_stopwords):
    sentences = nltk.sent_tokenize(message) # split the email message into sentences
    
    tokens = []
    for sentence in sentences:
        sentence_tokens = nltk.word_tokenize(sentence) # split each sentence into a list of words
        for token in sentence_tokens:
            if token.lower() not in en_stopwords:
                tokens.append(token.lower())
        
    return tokens

 ### $\Omega$ (Practice) Apply the new `preprocess_stopwords_removed` function to `msg_train`
 
 Previously, we applied the basic `preprocess` function to get the list of tokens for each training message in `msg_train`. Now let's apply the improved preprocessing function `preprocess_stopwords_removed`.

In [ ]:
# TODO: apply the new preprocess_stopwords_removed function to each message in msg_train







### $\Omega$ (Practice) Generate new predictions using the new set of  `words_train` and  `words_test`

Hint: this would be a lot similar to the code in Section 7.1.3, but with `preprocess_stopwords_removed` applied to each message in `msg_test` as well.

In [ ]:
# TODO: generate new predictions using the new set of  `words_train` and  `words_test`







### $\Delta$ Evaluate your new model

Is it better or similar to the performance we got from applying improvement 1 only?

In [ ]:
# TODO: evaluate your new model by comparing the new set of predictions with y_test







